# Vector boson fusion in IDM

The final notebook is [Significance_countours.ipynb](./Significance_countours.ipynb) which is based on [Sensitivity_fit.ipynb](Sensitivity_fit.ipynb) which is based on [Sensitivity_plots.ipynb](Sensitivity_plots.ipynb) by JD

### Initialization

In [134]:
import sys
import subprocess
import numpy as np
import pandas as pd
##pip3 install pyslha
#import pyslha # not longer required
import tempfile
import os
import re
def grep(pattern,multilinestring):
    '''Grep replacement in python
    as in: $ echo $multilinestring | grep pattern
    dev: re.M is for multiline strings
    '''
    import re 
    grp=re.finditer('(.*)%s(.*)' %pattern, multilinestring,re.M)
    return '\n'.join([g.group(0) for g in grp])

def subprocess_line_by_line(*args,TRUST_ERRORS=True,**kwargs):
    '''
    Subprocess output line by line. Stop of error found when TRUST_ERRORS=True, and simply
    report wait method otherwise.
    
    The arguments are the same as for the Popen constructor.
    
    WARNING: Works only in Python 3
    
    See: https://stackoverflow.com/a/28319191/2268280 
    and: https://stackoverflow.com/a/17698359/2268280
    
    Example:
    
    subprocess_line_by_line('for i in $(seq 1 3);do echo $i; sleep 1;done',shell=True)
    '''
    
    if not TRUST_ERRORS:
        kwargs['stderr']=subprocess.PIPE
        
    kwargs['stdout']=subprocess.PIPE
    kwargs['bufsize']=1
    kwargs['universal_newlines']=True
    s=subprocess.Popen(*args,**kwargs)
    with s as p:
        for line in p.stdout:
            print(line, end='') # process line here
    
    if TRUST_ERRORS:
        if p.returncode != 0:
            raise subprocess.CalledProcessError(p.returncode, p.args)
    else:
        return s.wait()
    
##Main madGRAPH script:
def preamble(MHc=750,MH0=110,UFO_model='InertDoublet_UFO',processes='generate p p > e+ e-'):
    return '''import model '''+UFO_model+'''
define p = g u c d s b u~ c~ d~ s~ b~
define j = p  
define l+ = e+ mu+ 
define l- = e- mu- 
define vl = ve vm vt 
define vl~ = ve~ vm~ vt~

'''+processes+'''

output ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL

'''

def lamL_loop(cfg,MHc=750,MH0=110,lamL=0.01):
    return '''launch ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL
0    
../'''+cfg.LHA_input_file+'''
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL '''+str(lamL)+'''
set mmh0 '''+str(MH0)+'''
set mma0 '''+str(MHc)+'''
set mmhch '''+str(MHc)+'''
0

''' 

def closing(MHc=750,MH0=110):
    return '''launch ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL -i
print_results --path=./result_BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lamdaL_mh0_110.txt --format=short


done
'''

def check_root_install(cfg):
    cfg=pd.Series(cfg)
    f=open('kk.sh','w')
    f.write('source '+cfg.thisroot+'\n')
    f.write('which root\n')
    f.close()
    
    if not subprocess.Popen('bash kk.sh'.split(),
                stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()[0]:
        sys.exit('INSTALL ROOT: see instrucctions in notebook')
        
def clone_git_repo(cfg,REPO='VBF_IDM',REPO_url='git@github.com:restrepo',git_options='--recursive',main_dir='.'):
    cfg=pd.Series(cfg)
    if cfg.CLONE_GIT_REPO:  
        REPO
        REPO_url
         #WARNING: Try to overwirte contents
        if os.path.exists(main_dir+'index.ipynb'):
            sys.exit('ERROR: Repo files already exists. Check main_dir')
        if not os.path.isdir(main_dir):
            s=subprocess.Popen(['mkdir','-p',main_dir],stdout=subprocess.PIPE,stderr=subprocess.PIPE).wait()

        td=tempfile.mkdtemp()
        s=subprocess_line_by_line(('git clone  '+git_options+' '+REPO_url+'/'+REPO+'.git').split(),cwd=td,
                     stdout=subprocess.PIPE,stderr=subprocess.PIPE,TRUST_ERRORS=False)

        s=subprocess.Popen('mv '+td+'/'+REPO+'/*  '+main_dir,shell=True,
                       stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

        s=subprocess.Popen('mv '+td+'/'+REPO+'/.* '+main_dir,shell=True,
                       stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

        os.rmdir(td+'/'+REPO)
        os.rmdir(td)
    else:
        if cfg.VERBOSE:
            print('Skiping git clone')        
            
def install_pythia_delphes(cfg,release='v2.3.3'):
    if cfg.INSTALL:
        s=subprocess.Popen('git branch'.split(),cwd=cfg.MADGRAPH,
                              stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

        if not grep('\* '+release,s[0].decode('utf-8')):
            s=subprocess.Popen( ('git checkout -b '+release).split(),cwd=cfg.MADGRAPH,
                              stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()
            if 'Switched' not in s[1].decode('utf-8'):
                sys.exit('Submodule problems')

        #subprocess does not use .bashrc        
        f=open(cfg.MADGRAPH+'/kk.sh','w')
        f.write('source '+cfg.thisroot+'\n')
        f.write('./bin/mg5_aMC install.dat\n')
        f.close()        
        if cfg.VERBOSE:
            subprocess_line_by_line('bash kk.sh'.split(),cwd=cfg.MADGRAPH, TRUST_ERRORS=False )
        else:
            s=subprocess.call('bash kk.sh'.split(),cwd=cfg.MADGRAPH, stdout=open('kk','w'),stderr=open('kkk','w') )
    else:
        if cfg.VERBOSE:
            print('Pythia: OK')
            print('Delphes: OK')  
                        
def not_html_opening(cfg):
    f=open(cfg.MADGRAPH+'/input/mg5_configuration.txt','r')
    mgc=f.read()
    f.close()

    f=open(cfg.MADGRAPH+'/input/mg5_configuration.txt','w')
    f.write(mgc.replace('# automatic_html_opening = True','automatic_html_opening = False'))
    f.close()           
            

def run_madgraph(cfg,MH0,MHc,LambdasL):
    s=subprocess.Popen(['mkdir','-p',cfg.work_dir],
                     stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()
    
    f=open(cfg.work_dir+'/'+cfg.work_script,'w')
    f.write( preamble(MHc,MH0,processes=cfg.processes) )
    for lamL in LambdasL:
        f.write( lamL_loop(cfg,MHc,MH0,lamL=lamL) )
    f.write( closing(MHc,MH0) )
    f.close()


    f=open(cfg.MADGRAPH+'/kk.sh','w')
    f.write('source '+cfg.thisroot+'\n')
    f.write('./bin/mg5_aMC ../'+cfg.work_dir+'/'+cfg.work_script+'\n')
    f.close()

    if not cfg.VERBOSE:
        s=subprocess.Popen( 'bash kk.sh'.split(), cwd=cfg.MADGRAPH,
                                        stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        s.wait()

    if cfg.VERBOSE:
        subprocess_line_by_line( 'bash kk.sh'.split(), cwd=cfg.MADGRAPH,TRUST_ERRORS=False)
        
def run_madevent(cfg,LambdasL):
    import pandas as pd
    if len(LambdasL)>99:
        sys.exit('ERROR: UPDATE FORMAT FOR > 99 runs')
    f=open(cfg.work_dir+'/'+cfg.pythia_script,'w')
    for r in range(1,len(LambdasL)+1):
        f.write('pythia run_%02d\n' %r)
        f.write('3\n')
        f.write('0\n')
    f.close()

    f=open(cfg.output_dir+'/kk.sh','w')
    f.write('source '+cfg.thisroot+'\n')
    f.write('./bin/madevent ../'+cfg.run_dir+'/'+cfg.pythia_script+'\n')
    f.close()

    (PHOUT,PHERR)=subprocess.Popen('bash kk.sh'.split(), cwd=cfg.output_dir,
                                    stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()

    #print(PHOUT.decode('utf-8'))

    cs_pb=np.array( re.sub( '\s+\+\-\s+[0-9\+\-eE\.]+\s+pb','\n',  
              re.sub('\s+Cross-section\s+:\s+','' ,
              ''.join( grep('Cross-section',PHOUT.decode('utf-8')).split('\n') 
              ) )  ).strip().split('\n')  ).astype(float64)

    if len(cs_pb)==len(LambdasL):
        df=pd.DataFrame({'xs_'+str(int(MH0)):cs_pb,'laL':LambdasL})
        return df
    else:
        sys.exit('Error: missing cross section')
        return pd.Series()
    
def store_output(cfg):
    cfg.full_output_dir='output'
    s=subprocess.Popen(['mkdir', '-p',cfg.full_output_dir]).wait()

    #if 'xs_'+str(int(MH0)) not in df.columns.values:
    if True:
        for r in range(1,len(LambdasL)+1):
            nrun='%02d' %r
            nrun3='%03d' %r
            s=subprocess.Popen(['cp',cfg.output_dir+'/Events/run_'+nrun+'/tag_1_delphes_events.root', 
                                cfg.full_output_dir+'/delphes_events_'+str(int(MH0))+'_'+nrun3+'_.root'],
                                 stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            if s.wait()>0:
                sys.exit('Files not found')

def merge_cross_sections_results(df)
    df_full=pd.read_csv('Output_data.csv')
    dff=df_full.merge(df,on='laL',how='left').fillna(0)
    tmp=dff['laL']
    dff=dff.drop('laL',axis='columns')
    dff['laL']=tmp
    return dff  

def main(scan_par,*input_par,**cfg):
    '''
    Simulation of processes for each component of the scan_par list of the model through full chain 
    madgraph - Pythia -Delphes.
    
    Implemented model: Inert Doublet Model
    scan_par -> LambdaL list.
    input_par: List of the model parameters
        input_par[0] -> MH0
        input_par[0] -> MHc
        
    Check main?? for **cfg options. 
    
    Default options are returned
    '''
    LambdasL=scan_par
    MH0=input_par[0]
    MHc=input_par[1]
    #Default values
    cfg=pd.Series(cfg)
    if 'thisroot' not in cfg:
        cfg['thisroot']='/home/restrepo/prog/ROOT/root/bin/thisroot.sh'
        #cfg['thisroot']='/opt/root5/bin/thisroot.sh'
    if 'MADGRAPH' not in cfg:           
        cfg['MADGRAPH']='madgraph'
    if 'run_dir' not in cfg:
        cfg['run_dir']='Task_Asana'
    if 'work_dir' not in cfg:        
        cfg['work_dir']='studies/IDM/'+cfg.run_dir
    if 'work_script' not in cfg:       
        cfg['work_script']='BP_'+str(int(MHc))+'_A_'+str(int(MH0))+'.txt'
    if 'LHA_input_file' not in cfg:       
        cfg['LHA_input_file']='MadGraph_cards/benchmarks/param_card_template.dat'
    if 'UFO_model' not in cfg:
        cfg['UFO_model']='InertDoublet_UFO'
    if 'processes' not in cfg:        
        cfg['processes']='generate p p > h2 h2 j j @0'        
        #cfg['processes']='generate p p > h2 h2'
    if 'output_dir' not in cfg:        
        cfg['output_dir']='studies/IDM/BP_'+str(int(MHc))+'_'+str(int(MH0))+'_vs_lambdaL' #madGRAPH output
    if 'pythia_script' not in cfg:        
        cfg['pythia_script']='TemplateRunPythiaDelphes_all.dat'
    if 'full_output_dir' not in cfg:
        cfg['full_output_dir'] ='output'
    if 'cross_sections_csv' not in cfg:
        cfg['cross_sections_csv']='cs_'+str(int(MHc))+'_'+str(int(MH0))+'.csv'
    if 'CLONE_GIT_REPO' not in cfg:        
        cfg['CLONE_GIT_REPO']=False #WARNING: Try to overwrite current contents!
    if 'INSTALL' not in cfg:          
        cfg['INSTALL']=False # If True check full installation
    if 'TEST' not in cfg:      
        cfg['TEST']=False #take a long time
    if 'VERBOSE' not in cfg:      
        cfg['VERBOSE']=True #Print shell commands output line by line        

    print('========= Preparing run...====')
    check_root_install(cfg)
    clone_git_repo(cfg)
    install_pythia_delphes(cfg)
    not_html_opening(cfg)
    #return cfg
    
    print('========= Runnig MadGraph (shown here if VERBOSE=True ====')
    run_madgraph(cfg,MH0,MHc,LambdasL)
    print('========= Runnig MadEvent. This can take a long...====')
    df=run_madevent(cfg,LambdasL)
    
    print('========= Saving root and csv files in :===='+cfg.full_output_dir+'...')
            
    df.to_csv(cfg.full_output_dir+'/'+cfg.cross_sections_csv,index=False)
            
    store_output(cfg)
    
    print('======== CONGRATULATIONS for the successful runs =====')

    return cfg

In [ ]:
if __name__=='__main__':
    MHc=750
    MH0=240
    if MH0%1!=0:
        sys.exit('ERROR: MH0 must be integer')

    LambdasL=[0.01]#,0.02,0.05,0.07,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,1.0,3.0,5.0,7.0,10.0]
    
    cfg=main(LambdasL,MH0,MHc)


========= Preparing run...====
Skiping git clone
Pythia: OK
Delphes: OK
========= Runnig MadGraph (shown here if VERBOSE=True ====
************************************************************
*                                                          *
*                     W E L C O M E to                     *
*              M A D G R A P H 5 _ a M C @ N L O           *
*                                                          *
*                                                          *
*                 *                       *                *
*                   *        * *        *                  *
*                     * * * * 5 * * * *                    *
*                   *        * *        *                  *
*                 *                       *                *
*                                                          *
*         VERSION 2.3.3                 2015-10-25         *
*                                                          *
*    The MadGra

INFO: Crossed process found for u d~ > h2 h2 u d~, reuse diagrams. 
INFO: Crossed process found for u d~ > h2 h2 u s~, reuse diagrams. 
INFO: Crossed process found for u d~ > h2 h2 u b~, reuse diagrams. 
INFO: Crossed process found for u d~ > h2 h2 c d~, reuse diagrams. 
INFO: Crossed process found for u d~ > h2 h2 c s~, reuse diagrams. 
INFO: Crossed process found for u d~ > h2 h2 c b~, reuse diagrams. 
INFO: Crossed process found for u s~ > h2 h2 u d~, reuse diagrams. 
INFO: Crossed process found for u s~ > h2 h2 u s~, reuse diagrams. 
INFO: Crossed process found for u s~ > h2 h2 u b~, reuse diagrams. 
INFO: Crossed process found for u s~ > h2 h2 c d~, reuse diagrams. 
INFO: Crossed process found for u s~ > h2 h2 c s~, reuse diagrams. 
INFO: Crossed process found for u s~ > h2 h2 c b~, reuse diagrams. 
INFO: Crossed process found for u b~ > h2 h2 u d~, reuse diagrams. 
INFO: Crossed process found for u b~ > h2 h2 u s~, reuse diagrams. 
INFO: Crossed process found for u b~ > h2 h2 u b

INFO: Process s u > h2 h2 u b added to mirror process u s > h2 h2 u b 
INFO: Process s u > h2 h2 c d added to mirror process u s > h2 h2 c d 
INFO: Process s u > h2 h2 c s added to mirror process u s > h2 h2 c s 
INFO: Process s u > h2 h2 c b added to mirror process u s > h2 h2 c b 
INFO: Process s c > h2 h2 u d added to mirror process c s > h2 h2 u d 
INFO: Process s c > h2 h2 u s added to mirror process c s > h2 h2 u s 
INFO: Process s c > h2 h2 u b added to mirror process c s > h2 h2 u b 
INFO: Process s c > h2 h2 c d added to mirror process c s > h2 h2 c d 
INFO: Process s c > h2 h2 c s added to mirror process c s > h2 h2 c s 
INFO: Process s c > h2 h2 c b added to mirror process c s > h2 h2 c b 
INFO: Process s d > h2 h2 d s added to mirror process d s > h2 h2 d s 
INFO: Trying process: s s > h2 h2 d d WEIGHTED=4  
INFO: Trying process: s s > h2 h2 d s WEIGHTED=4  
INFO: Trying process: s s > h2 h2 d b WEIGHTED=4  
INFO: Trying process: s s > h2 h2 s s WEIGHTED=4  
INFO: Process h

INFO: Crossed process found for u~ u~ > h2 h2 u~ u~, reuse diagrams. 
INFO: Crossed process found for u~ c~ > h2 h2 u~ c~, reuse diagrams. 
INFO: Crossed process found for u~ d~ > h2 h2 u~ d~, reuse diagrams. 
INFO: Crossed process found for u~ d~ > h2 h2 u~ s~, reuse diagrams. 
INFO: Crossed process found for u~ d~ > h2 h2 u~ b~, reuse diagrams. 
INFO: Crossed process found for u~ d~ > h2 h2 c~ d~, reuse diagrams. 
INFO: Crossed process found for u~ d~ > h2 h2 c~ s~, reuse diagrams. 
INFO: Crossed process found for u~ d~ > h2 h2 c~ b~, reuse diagrams. 
INFO: Crossed process found for u~ s~ > h2 h2 u~ d~, reuse diagrams. 
INFO: Crossed process found for u~ s~ > h2 h2 u~ s~, reuse diagrams. 
INFO: Crossed process found for u~ s~ > h2 h2 u~ b~, reuse diagrams. 
INFO: Crossed process found for u~ s~ > h2 h2 c~ d~, reuse diagrams. 
INFO: Crossed process found for u~ s~ > h2 h2 c~ s~, reuse diagrams. 
INFO: Crossed process found for u~ s~ > h2 h2 c~ b~, reuse diagrams. 
INFO: Crossed proces

INFO: Crossed process found for d~ d~ > h2 h2 d~ d~, reuse diagrams. 
INFO: Crossed process found for d~ s~ > h2 h2 d~ s~, reuse diagrams. 
INFO: Crossed process found for d~ b~ > h2 h2 d~ b~, reuse diagrams. 
INFO: Process s~ g > h2 h2 g s~ added to mirror process g s~ > h2 h2 g s~ 
INFO: Process s~ u > h2 h2 u d~ added to mirror process u s~ > h2 h2 u d~ 
INFO: Process s~ u > h2 h2 u s~ added to mirror process u s~ > h2 h2 u s~ 
INFO: Process s~ u > h2 h2 u b~ added to mirror process u s~ > h2 h2 u b~ 
INFO: Process s~ u > h2 h2 c d~ added to mirror process u s~ > h2 h2 c d~ 
INFO: Process s~ u > h2 h2 c s~ added to mirror process u s~ > h2 h2 c s~ 
INFO: Process s~ u > h2 h2 c b~ added to mirror process u s~ > h2 h2 c b~ 
INFO: Process s~ c > h2 h2 u d~ added to mirror process c s~ > h2 h2 u d~ 
INFO: Process s~ c > h2 h2 u s~ added to mirror process c s~ > h2 h2 u s~ 
INFO: Process s~ c > h2 h2 u b~ added to mirror process c s~ > h2 h2 u b~ 
INFO: Process s~ c > h2 h2 c d~ added to

...

INFO: Generating Feynman diagrams for Process: b u~ > h2 h2 b c~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group bux_h2h2bcx 
INFO: Creating files in directory P0_bcx_h2h2dux 
INFO: Generating Feynman diagrams for Process: b c~ > h2 h2 d u~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group bcx_h2h2dux 
INFO: Creating files in directory P0_bcx_h2h2dcx 
INFO: Generating Feynman diagrams for Process: b c~ > h2 h2 d c~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group bcx_h2h2dcx 
INFO: Creating files in directory P0_bcx_h2h2sux 
INFO: Generating Feynman diagrams for Process: b c~ > h2 h2 s u~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group bcx_h2h2sux 
INFO: Creating files in directory P0_bcx_h2h2scx 
INFO: Generating Feynman diagrams for Process: b c~ > h2 h2 s c~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group bcx_h2h2scx 
INFO: Creating files in directory P0_bcx_h2h2bux 
INFO: Generating Feynman diagr

INFO: get decay diagram for h+ 
INFO:     current estimated error: 0.152608609356 go to 4-body decay: 
Pass to numerical integration for computing the widths:
INFO: More info in temporary files:
    /home/restrepo/prog/2017/VBF_IDM/madgraph/tmpzYLSzT/temp_decay/index.html 
Results written to /home/restrepo/prog/2017/VBF_IDM/studies/IDM/BP_750_240_vs_lambdaL/Cards/param_card.dat
compile Source Directory
Using random number seed offset = 21
INFO: Running Survey 
Creating Jobs
Working on SubProcesses
INFO:     P0_gg_h2h2gg  
INFO:     P0_uu_h2h2uu  
INFO:     P0_cc_h2h2cc  
INFO:     P0_dd_h2h2dd  
INFO:     P0_ss_h2h2ss  
INFO:     P0_bb_h2h2bb  
INFO:     P0_ud_h2h2ud  
INFO:     P0_us_h2h2us  
INFO:     P0_ub_h2h2ub  
INFO:     P0_uux_h2h2ddx  
INFO:     P0_uux_h2h2ssx  
INFO:     P0_uux_h2h2bbx  
INFO:     P0_cd_h2h2cd  
INFO:     P0_cs_h2h2cs  
INFO:     P0_cb_h2h2cb  
INFO:     P0_ccx_h2h2ddx  
INFO:     P0_ccx_h2h2ssx  
INFO:     P0_ccx_h2h2bbx  
INFO:     P0_dux_h2h2dux  
INFO:   

INFO:  Idle: 1237,  Running: 4,  Completed: 189 [  6m 32s  ] 
INFO:  Idle: 1234,  Running: 4,  Completed: 192 [  6m 38s  ] 
INFO:  Idle: 1233,  Running: 4,  Completed: 193 [  6m 44s  ] 
INFO:  Idle: 1232,  Running: 4,  Completed: 194 [  6m 49s  ] 
INFO:  Idle: 1231,  Running: 4,  Completed: 195 [  6m 52s  ] 
INFO:  Idle: 1230,  Running: 4,  Completed: 196 [  6m 58s  ] 
INFO:  Idle: 1228,  Running: 4,  Completed: 198 [  7m 4s  ] 
INFO:  Idle: 1227,  Running: 4,  Completed: 199 [  7m 18s  ] 
INFO:  Idle: 1226,  Running: 4,  Completed: 200 [  7m 21s  ] 
INFO:  Idle: 1225,  Running: 4,  Completed: 201 [  7m 26s  ] 
INFO:  Idle: 1224,  Running: 4,  Completed: 202 [  7m 56s  ] 
INFO:  Idle: 1223,  Running: 4,  Completed: 203 [  8m 1s  ] 
INFO:  Idle: 1222,  Running: 4,  Completed: 204 [  8m 8s  ] 
INFO:  Idle: 1221,  Running: 4,  Completed: 205 [  8m 12s  ] 
INFO:  Idle: 1220,  Running: 4,  Completed: 206 [  8m 27s  ] 
INFO:  Idle: 1219,  Running: 4,  Completed: 207 [  8m 31s  ] 
INFO:  Idle

INFO:  Idle: 986,  Running: 4,  Completed: 440 [  25m 24s  ] 
INFO:  Idle: 984,  Running: 4,  Completed: 442 [  25m 29s  ] 
INFO:  Idle: 981,  Running: 4,  Completed: 445 [  25m 33s  ] 
INFO:  Idle: 977,  Running: 4,  Completed: 449 [  25m 38s  ] 
INFO:  Idle: 973,  Running: 4,  Completed: 453 [  25m 42s  ] 
INFO:  Idle: 968,  Running: 4,  Completed: 458 [  25m 55s  ] 
INFO:  Idle: 967,  Running: 4,  Completed: 459 [  25m 59s  ] 
INFO:  Idle: 963,  Running: 4,  Completed: 463 [  26m 2s  ] 
INFO:  Idle: 962,  Running: 4,  Completed: 464 [  26m 6s  ] 
INFO:  Idle: 959,  Running: 4,  Completed: 467 [  26m 9s  ] 
INFO:  Idle: 958,  Running: 4,  Completed: 468 [  26m 26s  ] 
INFO:  Idle: 957,  Running: 4,  Completed: 469 [  26m 42s  ] 
INFO:  Idle: 955,  Running: 4,  Completed: 471 [  26m 49s  ] 
INFO:  Idle: 953,  Running: 4,  Completed: 473 [  26m 53s  ] 
INFO:  Idle: 951,  Running: 4,  Completed: 475 [  26m 58s  ] 
INFO:  Idle: 948,  Running: 4,  Completed: 478 [  27m 2s  ] 
INFO:  Idle:

INFO:  Idle: 714,  Running: 4,  Completed: 712 [  45m 14s  ] 
INFO:  Idle: 710,  Running: 4,  Completed: 716 [  45m 51s  ] 
INFO:  Idle: 708,  Running: 4,  Completed: 718 [  46m 1s  ] 
INFO:  Idle: 707,  Running: 4,  Completed: 719 [  46m 9s  ] 
INFO:  Idle: 704,  Running: 4,  Completed: 722 [  46m 13s  ] 
INFO:  Idle: 702,  Running: 4,  Completed: 724 [  46m 16s  ] 
INFO:  Idle: 701,  Running: 4,  Completed: 725 [  46m 19s  ] 
INFO:  Idle: 699,  Running: 4,  Completed: 727 [  46m 23s  ] 
INFO:  Idle: 696,  Running: 4,  Completed: 730 [  46m 26s  ] 
INFO:  Idle: 695,  Running: 4,  Completed: 731 [  46m 48s  ] 
INFO:  Idle: 694,  Running: 4,  Completed: 732 [  47m 6s  ] 
INFO:  Idle: 693,  Running: 4,  Completed: 733 [  47m 13s  ] 


In [ ]:
#def test_all():
if True:
    from nose.tools import assert_equal
    MHc=750
    MH0=240
    LambdasL=[0.01]
    
    cfg=main(LambdasL,MH0,MHc,work_script='test.txt',processes='generate p p > h2 h2',
            output_dir='test/tmp',full_output_dir='test',cross_sections_csv='test.csv',VERBOSE=False)
    df=pd.read_csv(full_output_dir+'/'+cross_sections_csv)
    assert_equal(df.xs_240.values[0],5.288E-8)
#def test_full_install
#def test_pythia_delphes_install

In [153]:
cfg

thisroot                  /home/restrepo/prog/ROOT/root/bin/thisroot.sh
MADGRAPH                                                       madgraph
run_dir                                                      Task_Asana
work_dir                                         studies/IDM/Task_Asana
work_script                                            BP_750_A_240.txt
LHA_input_file        MadGraph_cards/benchmarks/param_card_template.dat
UFO_model                                              InertDoublet_UFO
processes                                          generate p p > h2 h2
output_dir                            studies/IDM/BP_750_240_vs_lambdaL
pythia_script                          TemplateRunPythiaDelphes_all.dat
full_output_dir                                                  output
cross_sections_csv                                       cs_750_240.csv
CLONE_GIT_REPO                                                    False
INSTALL                                                         

## Appendix: Install root 5

### Prerequisites 
```bash
apt-get install cmake git dpkg-dev make g++ gcc binutils libx11-dev libxpm-dev \
libxft-dev libxext-dev gfortran libssl-dev libpcre3-dev \
xlibmesa-glu-dev libglew1.5-dev libftgl-dev \
libmysqlclient-dev libfftw3-dev libcfitsio-dev \
graphviz-dev libavahi-compat-libdnssd-dev \
libldap2-dev python-dev python3-dev libxml2-dev libkrb5-dev \
libgsl0-dev libqt4-dev r-base r-base-dev
```

Install ROOT 5 in some `PATH`
```bash
git clone http://root.cern.ch/git/root.git
cd root
git checkout v5-34-00-patches
./configure
make

```
Add to your `.bashrc`
```bash
source PATH/root/bin/thisroot.sh
```

##  Appendix: File results

```bash
rsiii@gfif:~/IDM_VBF/Samples/mh0_vs_lal/scan$ ls delphes_events_[0-9][0-9]_*.root | more
delphes_events_70_001_.root
delphes_events_70_002_.root
delphes_events_70_003_.root
...
...
delphes_events_90_018_.root
delphes_events_90_019_.root
delphes_events_90_020_.root
```

```bash
rsiii@gfif:~/IDM_VBF/Samples/mh0_vs_lal/scan$ ls delphes_events_[0-9][0-9][0-9]_*.root | more
delphes_events_110_001_.root
delphes_events_110_002_.root
delphes_events_110_003_.root
...
...
delphes_events_240_018_.root
delphes_events_240_019_.root
delphes_events_240_020_.root
```

cat readme.txt 
date 23/01/2017
Delphes output (.root) for the set the points contained in data_mh0_vs_x.dat

The data is organized as follows (in ascending order in lambda_L[lal] ):

#lal #mh0[GeV] #xs[fb]   #Delphes name
0.3 63.75 200        --> delphes_events_2.root
0.4 64.80 200        --> delphes_events_3.root 
0.5 66.32 200        --> delphes_events_4.root
0.6 68.05 200        --> delphes_events_5.root 
0.7 70.04 200        --> delphes_events_6.root
.   .     .          --> 
.   .     .          --> 
.   .     .          -->

The cross-section (xs) for the processes p p > h0 h0 j j
for all the set of points is around 200 fb. [Except for
the first point, with lal=0.3,  where the cross-section turns out to be 179 fb.]